In [1]:
import pandas as pd
import statsmodels.formula.api as smf

# Data Cleaning

In [2]:
weather = pd.read_csv('weather_data_nyc_centralpark_2016.csv')
weather.head()

,date,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth
0,1-1-2016,42,34,38.0,0.00,0.0,0
1,2-1-2016,40,32,36.0,0.00,0.0,0
2,3-1-2016,45,35,40.0,0.00,0.0,0
3,4-1-2016,36,14,25.0,0.00,0.0,0
4,5-1-2016,29,11,20.0,0.00,0.0,0


In [3]:
w = weather[['date','average temperature','precipitation']]
w.head()

,date,average temperature,precipitation
0,1-1-2016,38.0,0.00
1,2-1-2016,36.0,0.00
2,3-1-2016,40.0,0.00
3,4-1-2016,25.0,0.00
4,5-1-2016,20.0,0.00


In [4]:
data = pd.read_csv('JoinedEvent2_CB2016_subset_800m.csv')
data.head()

,Unnamed: 0,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,...,usertype,birth year,gender,startdate,stopdate,Event_type,End_Time,weekday,O_date,O_hour
0,0,173,16:03:10,16:06:03,243,Fulton St & Rockwell Pl,40.688226,-73.979382,241,DeKalb Ave & S Portland Ave,...,Subscriber,1971.0,2,2016-01-01,2016-01-01,no-event,NaN,NaN,2016-01-01,16
1,1,136,16:05:54,16:08:11,420,Clermont Ave & Lafayette Ave,40.687645,-73.969689,270,Adelphi St & Myrtle Ave,...,Subscriber,1980.0,1,2016-01-01,2016-01-01,no-event,NaN,NaN,2016-01-01,16
2,2,653,16:13:47,16:24:40,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,278,Concord St & Bridge St,...,Subscriber,1976.0,1,2016-01-01,2016-01-01,no-event,NaN,NaN,2016-01-01,16
3,3,659,16:13:47,16:24:46,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,278,Concord St & Bridge St,...,Subscriber,1985.0,2,2016-01-01,2016-01-01,no-event,NaN,NaN,2016-01-01,16
4,4,1419,16:20:39,16:44:19,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,532,S 5 Pl & S 4 St,...,Subscriber,1993.0,1,2016-01-01,2016-01-01,no-event,NaN,NaN,2016-01-01,16


In [5]:
d = pd.DataFrame(data.groupby(['O_date','Event_type','O_hour'],as_index=False).size())
d.reset_index(inplace=True)
d.columns = ['O_date','Event_type','O_hour','Count']
d.head()

,O_date,Event_type,O_hour,Count
0,2016-01-01,no-event,16,8
1,2016-01-01,no-event,17,7
2,2016-01-01,no-event,18,4
3,2016-01-01,no-event,19,2
4,2016-01-01,no-event,20,2


# Data Merging

In [6]:
w['date'] = pd.to_datetime(w.date)
d['O_date'] = pd.to_datetime(d.O_date)
d = d.merge(w,left_on='O_date',right_on='date',how='left')

/Users/Yuwen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
d.rename(columns={'average temperature':'temperature'},inplace=True)

# Data Engineering

In [8]:
# modify precipitation data type
d['precipitation'] = pd.to_numeric(d['precipitation'],errors='coerce')
d['precipitation'][d['precipitation'].isnull()] = 0
d.describe()

/Users/Yuwen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,O_hour,Count,temperature,precipitation
count,2879.000000,2879.000000,2879.000000,2879.000000
mean,19.489406,20.891976,57.619660,0.109851
std,2.288399,17.545349,16.906403,0.289087
min,16.000000,1.000000,7.000000,0.000000
25%,17.000000,9.000000,44.500000,0.000000
50%,19.000000,16.000000,56.500000,0.000000
75%,21.000000,27.000000,74.000000,0.040000
max,23.000000,109.000000,88.500000,2.200000


In [9]:
# generate weekday, month, season
d['weekday'] = [i not in [5,6] for i in d['O_date'].dt.weekday.values]
d['O_month'] = pd.to_datetime(d['O_date']).dt.month
d['season'] = d['O_month'].map({1: 1,
                              2: 1,
                              3: 2,
                              4: 2,
                              5: 2,
                              6: 3,
                              7: 3,
                              8: 3,
                              9: 4,
                              10: 4,
                              11: 4,
                              12: 1})
d.head()

,O_date,Event_type,O_hour,Count,date,temperature,precipitation,weekday,O_month,season
0,2016-01-01,no-event,16,8,2016-01-01,38.0,0.0,True,1,1
1,2016-01-01,no-event,17,7,2016-01-01,38.0,0.0,True,1,1
2,2016-01-01,no-event,18,4,2016-01-01,38.0,0.0,True,1,1
3,2016-01-01,no-event,19,2,2016-01-01,38.0,0.0,True,1,1
4,2016-01-01,no-event,20,2,2016-01-01,38.0,0.0,True,1,1


# Regression Analysis
## All records (16-24)

In [10]:
# previous best model
lm5 = smf.ols(formula="Count ~ weekday + season * O_hour * C(Event_type, Treatment(reference='no-event'))",data=d).fit()
lm5.summary()

# f = open('lm5.txt', 'w')
# f.write(lm5.summary().as_text())
# f.close()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Count   R-squared:                       0.480
Model:                            OLS   Adj. R-squared:                  0.475
Method:                 Least Squares   F-statistic:                     93.93
Date:                Sat, 09 Dec 2017   Prob (F-statistic):               0.00
Time:                        19:52:00   Log-Likelihood:                -11391.
No. Observations:                2879   AIC:                         2.284e+04
Df Residuals:                    2850   BIC:                         2.301e+04
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
==============================================================================================================================================
                                                                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                    -17.3744      6.823     -2.547      0.011     -30.752      -3.996
weekday[T.True]                                                                1.3571      0.545      2.489      0.013       0.288       2.426
C(Event_type, Treatment(reference='no-event'))[T.basketball]                  30.7881     12.807      2.404      0.016       5.677      55.900
C(Event_type, Treatment(reference='no-event'))[T.boxing]                      84.3749     35.322      2.389      0.017      15.116     153.634
C(Event_type, Treatment(reference='no-event'))[T.concert]                    -27.8251     22.024     -1.263      0.207     -71.010      15.359
C(Event_type, Treatment(reference='no-event'))[T.family]                      88.5216     51.958      1.704      0.089     -13.357     190.400
C(Event_type, Treatment(reference='no-event'))[T.hockey]                      77.9004     18.918      4.118      0.000      40.805     114.996
C(Event_type, Treatment(reference='no-event'))[T.other]                       20.3641     40.326      0.505      0.614     -58.706      99.434
season                                                                        31.7726      2.563     12.397      0.000      26.747      36.798
season:C(Event_type, Treatment(reference='no-event'))[T.basketball]          -10.8377      4.976     -2.178      0.029     -20.595      -1.080
season:C(Event_type, Treatment(reference='no-event'))[T.boxing]              -29.3775     14.353     -2.047      0.041     -57.520      -1.235
season:C(Event_type, Treatment(reference='no-event'))[T.concert]               8.9187      7.724      1.155      0.248      -6.226      24.063
season:C(Event_type, Treatment(reference='no-event'))[T.family]              -15.4198     13.655     -1.129      0.259     -42.194      11.354
season:C(Event_type, Treatment(reference='no-event'))[T.hockey]              -12.9605      6.028     -2.150      0.032     -24.781      -1.141
season:C(Event_type, Treatment(reference='no-event'))[T.other]                -1.9350     13.196     -0.147      0.883     -27.809      23.939
O_hour                                                                         0.6494      0.348      1.868      0.062      -0.032       1.331
O_hour:C(Event_type, Treatment(reference='no-event'))[T.basketball]           -1.1641      0.653     -1.783      0.075      -2.444       0.116
O_hour:C(Event_type, Treatment(reference='no-event'))[T.boxing]               -3.5239      1.804     -1.953      0.051      -7.062       0.014
O_hour:C(Event_type, Treatment(reference='no-event'))[T.concert]               1

In [12]:
# add weather vars
lm6 = smf.ols(formula="Count ~ weekday + precipitation + temperature + season * O_hour * C(Event_type, Treatment(reference='no-event'))",data=d).fit()
lm6.summary()

# f = open('lm6.txt', 'w')
# f.write(lm6.summary().as_text())
# f.close()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Count   R-squared:                       0.487
Model:                            OLS   Adj. R-squared:                  0.482
Method:                 Least Squares   F-statistic:                     90.24
Date:                Sat, 09 Dec 2017   Prob (F-statistic):               0.00
Time:                        19:52:24   Log-Likelihood:                -11371.
No. Observations:                2879   AIC:                         2.280e+04
Df Residuals:                    2848   BIC:                         2.299e+04
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
==============================================================================================================================================
                                                                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                    -15.8495      6.803     -2.330      0.020     -29.189      -2.510
weekday[T.True]                                                                1.3963      0.542      2.578      0.010       0.334       2.458
C(Event_type, Treatment(reference='no-event'))[T.basketball]                  30.0250     12.722      2.360      0.018       5.080      54.970
C(Event_type, Treatment(reference='no-event'))[T.boxing]                      84.2040     35.082      2.400      0.016      15.415     152.993
C(Event_type, Treatment(reference='no-event'))[T.concert]                    -28.7988     21.876     -1.316      0.188     -71.692      14.095
C(Event_type, Treatment(reference='no-event'))[T.family]                      88.4005     51.608      1.713      0.087     -12.792     189.593
C(Event_type, Treatment(reference='no-event'))[T.hockey]                      77.6330     18.791      4.131      0.000      40.788     114.478
C(Event_type, Treatment(reference='no-event'))[T.other]                       19.0004     40.055      0.474      0.635     -59.539      97.540
precipitation                                                                 -5.1557      0.834     -6.182      0.000      -6.791      -3.520
temperature                                                                   -0.0229      0.015     -1.506      0.132      -0.053       0.007
season                                                                        31.9443      2.548     12.536      0.000      26.948      36.941
season:C(Event_type, Treatment(reference='no-event'))[T.basketball]          -10.7128      4.944     -2.167      0.030     -20.407      -1.019
season:C(Event_type, Treatment(reference='no-event'))[T.boxing]              -29.1979     14.255     -2.048      0.041     -57.149      -1.246
season:C(Event_type, Treatment(reference='no-event'))[T.concert]               9.0402      7.671      1.178      0.239      -6.002      24.082
season:C(Event_type, Treatment(reference='no-event'))[T.family]              -15.4309     13.563     -1.138      0.255     -42.025      11.163
season:C(Event_type, Treatment(reference='no-event'))[T.hockey]              -12.9257      5.988     -2.159      0.031     -24.667      -1.185
season:C(Event_type, Treatment(reference='no-event'))[T.other]                -1.1091     13.108     -0.085      0.933     -26.811      24.592
O_hour                                                                         0.6515      0.345      1.887      0.059      -0.026       1.329
O_hour:C(Event_type, Treatment(reference='no-event'))[T.basketball]           -1

## Subset records (19-24)

In [14]:
d = d[d['O_hour']>18]

In [15]:
lm7 = smf.ols(formula="Count ~ weekday + precipitation + temperature + season * O_hour * C(Event_type, Treatment(reference='no-event'))",data=d).fit()
lm7.summary()

# f = open('lm3.txt', 'w')
# f.write(lm3.summary().as_text())
# f.close()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Count   R-squared:                       0.592
Model:                            OLS   Adj. R-squared:                  0.585
Method:                 Least Squares   F-statistic:                     85.30
Date:                Sat, 09 Dec 2017   Prob (F-statistic):          6.51e-317
Time:                        19:52:53   Log-Likelihood:                -6542.0
No. Observations:                1795   AIC:                         1.315e+04
Df Residuals:                    1764   BIC:                         1.332e+04
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
==============================================================================================================================================
                                                                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                    -32.4875     11.082     -2.931      0.003     -54.224     -10.751
weekday[T.True]                                                                4.7082      0.508      9.272      0.000       3.712       5.704
C(Event_type, Treatment(reference='no-event'))[T.basketball]                  50.2078     20.680      2.428      0.015       9.648      90.767
C(Event_type, Treatment(reference='no-event'))[T.boxing]                     103.5489     56.704      1.826      0.068      -7.665     214.763
C(Event_type, Treatment(reference='no-event'))[T.concert]                    -53.4284     35.489     -1.505      0.132    -123.034      16.177
C(Event_type, Treatment(reference='no-event'))[T.family]                     112.7382     83.862      1.344      0.179     -51.742     277.218
C(Event_type, Treatment(reference='no-event'))[T.hockey]                      80.0080     30.556      2.618      0.009      20.079     139.937
C(Event_type, Treatment(reference='no-event'))[T.other]                        4.9927     65.092      0.077      0.939    -122.673     132.659
precipitation                                                                 -3.7322      0.781     -4.782      0.000      -5.263      -2.201
temperature                                                                    0.0079      0.014      0.556      0.578      -0.020       0.036
season                                                                        53.5440      4.155     12.886      0.000      45.395      61.694
season:C(Event_type, Treatment(reference='no-event'))[T.basketball]          -19.0285      8.036     -2.368      0.018     -34.790      -3.267
season:C(Event_type, Treatment(reference='no-event'))[T.boxing]              -57.9358     23.080     -2.510      0.012    -103.204     -12.668
season:C(Event_type, Treatment(reference='no-event'))[T.concert]              13.9316     12.450      1.119      0.263     -10.487      38.351
season:C(Event_type, Treatment(reference='no-event'))[T.family]              -28.7579     22.040     -1.305      0.192     -71.985      14.470
season:C(Event_type, Treatment(reference='no-event'))[T.hockey]              -11.4470      9.736     -1.176      0.240     -30.543       7.649
season:C(Event_type, Treatment(reference='no-event'))[T.other]               -16.7088     21.300     -0.784      0.433     -58.484      25.066
O_hour                                                                         1.2847      0.526      2.442      0.015       0.253       2.317
O_hour:C(Event_type, Treatment(reference='no-event'))[T.basketball]           -2

## Subset records (w/o other events)

In [16]:
print(len(d))
d = d[d['Event_type'] != 'other']
print(len(d))

1795
1760


In [17]:
lm8 = smf.ols(formula="Count ~ weekday + precipitation + temperature + season * O_hour * C(Event_type, Treatment(reference='no-event'))",data=d).fit()
lm8.summary()

# f = open('lm3.txt', 'w')
# f.write(lm3.summary().as_text())
# f.close()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Count   R-squared:                       0.592
Model:                            OLS   Adj. R-squared:                  0.586
Method:                 Least Squares   F-statistic:                     96.74
Date:                Sat, 09 Dec 2017   Prob (F-statistic):          2.90e-314
Time:                        19:53:01   Log-Likelihood:                -6422.6
No. Observations:                1760   AIC:                         1.290e+04
Df Residuals:                    1733   BIC:                         1.305e+04
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
==============================================================================================================================================
                                                                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                    -32.5624     11.123     -2.927      0.003     -54.379     -10.746
weekday[T.True]                                                                4.9146      0.515      9.540      0.000       3.904       5.925
C(Event_type, Treatment(reference='no-event'))[T.basketball]                  50.1976     20.756      2.418      0.016       9.489      90.907
C(Event_type, Treatment(reference='no-event'))[T.boxing]                     103.5443     56.912      1.819      0.069      -8.079     215.168
C(Event_type, Treatment(reference='no-event'))[T.concert]                    -53.4074     35.620     -1.499      0.134    -123.269      16.455
C(Event_type, Treatment(reference='no-event'))[T.family]                     112.7383     84.170      1.339      0.181     -52.347     277.824
C(Event_type, Treatment(reference='no-event'))[T.hockey]                      80.0174     30.668      2.609      0.009      19.868     140.167
precipitation                                                                 -3.5236      0.806     -4.369      0.000      -5.105      -1.942
temperature                                                                    0.0058      0.014      0.404      0.686      -0.022       0.034
season                                                                        53.5545      4.170     12.842      0.000      45.375      61.734
season:C(Event_type, Treatment(reference='no-event'))[T.basketball]          -19.0405      8.066     -2.361      0.018     -34.860      -3.221
season:C(Event_type, Treatment(reference='no-event'))[T.boxing]              -57.8950     23.165     -2.499      0.013    -103.329     -12.461
season:C(Event_type, Treatment(reference='no-event'))[T.concert]              13.9323     12.496      1.115      0.265     -10.577      38.441
season:C(Event_type, Treatment(reference='no-event'))[T.family]              -28.7516     22.121     -1.300      0.194     -72.138      14.635
season:C(Event_type, Treatment(reference='no-event'))[T.hockey]              -11.4573      9.772     -1.172      0.241     -30.624       7.709
O_hour                                                                         1.2853      0.528      2.434      0.015       0.250       2.321
O_hour:C(Event_type, Treatment(reference='no-event'))[T.basketball]           -2.1673      0.987     -2.195      0.028      -4.104      -0.231
O_hour:C(Event_type, Treatment(reference='no-event'))[T.boxing]               -4.5053      2.704     -1.666      0.096      -9.808       0.797
O_hour:C(Event_type, Treatment(reference='no-event'))[T.concert]               2